In [45]:
library("ComplexHeatmap")
library("ggplot2")
library("circlize")
library("qpdf")

Warning message:
“package ‘qpdf’ was built under R version 4.3.2”


In [2]:
source("/data1/weiyihu/endometrium/data/share/paraments.R")
source("/data1/weiyihu/endometrium/data/share/function.R")

In [3]:
setwd("/data1/weiyihu/endometrium/data")

In [4]:
dataHeatmap = readRDS("0006_dataHeatmap.rds")

In [37]:
makeHeatmap = function(df, rowSet=NULL, colSet=NULL, rowColor=NULL, colColor=NULL, rowLevels=NULL, colLevels=NULL){
  # input:
  #   df
  #   rowSet|\t     list|\t             key为row的分组名, value为对应分组中的rowName
  #   colSet|\t     list|\t             同上
  #   rowColor|\t   list|\t             key为row的组名, value为该组的颜色
  #   colColor|\t   list|\t             key为col的组名, value为该组的颜色
  #   rowLevels|\t  characterVector|\t  =names(rowSet), 行组的顺序
  #   colLevels|\t  characterVector|\t  =unique(colnames(df)), 列组的顺序
  #library(ComplexHeatmap)
  #library(circlize)
  #library(grDevices)
  df = df
  rowSet = rowSet
  rowColor = rowColor
  colColor = colColor
  rowLevels = rowLevels
  colLevels = colLevels

  # 初始化
  rowSet = ifelse(is.null(rowSet), list(structure(as.list(rownames(df)), names=rownames(df))), list(rowSet))[[1]]
  colSet = ifelse(is.null(colSet), list(structure(as.list(colnames(df)), names=colnames(df))), list(colSet))[[1]]
  rowColor = ifelse(is.null(rowColor), list(stats::setNames(grDevices::rainbow(length(rowSet)), names(rowSet))), list(unlist(rowColor)[names(rowColor) %in% names(rowSet)]))[[1]]
  colColor = ifelse(is.null(colColor), list(stats::setNames(grDevices::rainbow(length(colSet)), names(colSet))), list(unlist(colColor)[names(colColor) %in% names(colSet)]))[[1]]
  rowLevels = ifelse(is.null(rowLevels), list(names(rowSet)), list(rowLevels[rowLevels %in% names(rowSet)]))[[1]]
  colLevels = ifelse(is.null(colLevels), list(unique(colnames(df))), list(colLevels[colLevels %in% names(colSet)]))[[1]]

  df = as.data.frame(t(df))

  #顶部注释
  dfAnno = df
  dfAnno$column = rownames(dfAnno)
  dfAnno <- data.frame(dfAnno$column)
  dfAnno$group = NA
  for(i in seq(1, dim(dfAnno)[1])){
    colName = dfAnno[i, 1]
    for(group in names(colSet)){
      if(colName %in% colSet[[group]]){dfAnno[i, 2] = group}
    }
  }
  dfAnno = dfAnno[, 2, drop=FALSE]
  colnames(dfAnno) <- "group"
  dfAnno[, 1] = factor(dfAnno[, 1], levels=colLevels)
  splitCol = dfAnno[, 1]
  top_anno = HeatmapAnnotation(df=dfAnno,
                               border=F,
                               show_annotation_name=F,
                               gp=gpar(col=NA),
                               col=list("group"=colColor),
                               show_legend=T,
                               which='column')

  # 左侧注释
  dfAnno = df
  dfAnno = as.data.frame(t(dfAnno))
  dfAnno$rowName = rownames(dfAnno)
  dfAnno <- data.frame(dfAnno$rowName)
  dfAnno$group = NA
  for(i in seq(1, dim(dfAnno)[1])){
    rowName = dfAnno[i, 1]
    for(group in names(rowSet)){
      if(rowName %in% rowSet[[group]]){dfAnno[i, 2] = group}
    }
  }
  dfAnno = dfAnno[, 2, drop=FALSE]
  colnames(dfAnno) <- "gene set"
  dfAnno[, 1] = factor(dfAnno[, 1], levels=rowLevels)
  splitRow = dfAnno[, 1]
  left_anno = HeatmapAnnotation(df=dfAnno,
                                border=F,
                                show_annotation_name=F,
                                gp=gpar(col=NA),
                                col=list("gene set"=unlist(rowColor)),
                                show_legend=T,
                                which='row')

  # 数据标准化缩放
  marker_exp <- t(scale(df[, ], scale=T, center=T))
  cat(paste0("min(int): ", floor(min(marker_exp)), ' max(int): ', ceiling(max(marker_exp)), '\n'))
  
  #颜色设置
  col_fun <- colorRamp2(c(floor(min(marker_exp)), 0, ceiling(max(marker_exp))),
                        c("#313695", "white", "#A50026"))
  col_fun(seq(floor(min(marker_exp)), ceiling(max(marker_exp))))
  
  # 主图
plot <- Heatmap(marker_exp,
                  row_split=splitRow,
                  column_split=splitCol,
                  cluster_row_slices=FALSE,
                  cluster_column_slices=FALSE,
                  #cluster_rows=T,  # 是否进行行聚类
                  cluster_rows=F,  # 是否进行行聚类
                  show_row_dend=FALSE,
                  cluster_columns=F,  # 是否进行列聚类
                  row_title=gt_render(c("stromal cells"),r=unit(2, "pt"),padding=unit(c(2,2,2,2),"pt")), # nolint: commented_code_linter.
                  row_title_side="left",
                  show_row_names=T,  # 是否显示行名
                  row_names_side='right',  # 行名的位置
                  row_names_gp=gpar(fontsize=10),  # 行名的字体
                  show_column_names=F,  # 是否显示列名
                  column_title=NULL,  # 所有行的title
                  column_names_gp=gpar(fontsize=10),  # 列名的字体
                  heatmap_legend_param=list(title=' '),
                  col=col_fun,  # 色条的设置
                  border=NULL,  # 热图最外侧边界的颜色
                  top_annotation=top_anno,
                  left_annotation=left_anno
                  )

  return(plot)
}

In [46]:
plot1 = makeHeatmap(df=dataHeatmap$raw,
                    rowSet=geneSet[["stromal"]],
                    colSet=sampleInGroup[["group2"]],
                    rowColor=geneSetColors,
                    colColor=groupColors,
                    rowLevels=c('Angiogenesis','AdhesionMolecule','LymphocyteRecruitment'),
                    colLevels=levels(groupLevels[["group2"]])
                    )

min(int): -3 max(int): 11


In [47]:
plot2 = makeHeatmap(df=dataHeatmap$rawMean,
                    rowSet=geneSet[["stromal"]],
                    colSet=NULL,
                    rowColor=geneSetColors,
                    colColor=groupColors,
                    rowLevels=c('Angiogenesis','AdhesionMolecule','LymphocyteRecruitment'),
                    colLevels=levels(groupLevels[["group2"]])
                    )

min(int): -2 max(int): 2


In [48]:
plot3 = makeHeatmap(df=dataHeatmap$filter,
                    rowSet=geneSet[["stromal"]],
                    colSet=sampleInGroup[["group2"]],
                    rowColor=geneSetColors,
                    colColor=groupColors,
                    rowLevels=c('Angiogenesis','AdhesionMolecule','LymphocyteRecruitment'),
                    colLevels=levels(groupLevels[["group2"]])
                    )

min(int): -2 max(int): 11


In [49]:
plot4 = makeHeatmap(df=dataHeatmap$filterMean,
                    rowSet=geneSet[["stromal"]],
                    colSet=NULL,
                    rowColor=geneSetColors,
                    colColor=groupColors,
                    rowLevels=c('Angiogenesis','AdhesionMolecule','LymphocyteRecruitment'),
                    colLevels=levels(groupLevels[["group2"]])
                    )

min(int): -2 max(int): 2


In [50]:
pdf("0006_plot.pdf", width=10, height=10)
print(plot1)
print(plot2)
print(plot3)
print(plot4)
dev.off()

Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied colo

png 
  2

In [52]:
pdf("temp1.pdf", width=10, height=10)
plot1
dev.off()
pdf("temp2.pdf", width=10, height=10)
plot2
dev.off()
pdf("temp3.pdf", width=10, height=10)
plot3
dev.off()
pdf("temp4.pdf", width=10, height=10)
plot4
dev.off()
qpdf::pdf_combine(c("temp1.pdf", "temp2.pdf", "temp3.pdf", "temp4.pdf"),
                  "0006_plot.pdf")
file.remove(c("temp1.pdf", "temp2.pdf", "temp3.pdf", "temp4.pdf"))

Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied colo

png 
  2

Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied colo

png 
  2

Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied colo

png 
  2

Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied color is neither numeric nor character”
Warning message in grid.Call.graphics(C_rect, x$x, x$y, x$width, x$height, resolveHJust(x$just, :
“supplied colo

png 
  2

[1] "/data1/weiyihu/endometrium/data/0006_plot.pdf"

[1] TRUE TRUE TRUE TRUE